In [1]:
import os

# Add root directory to python path
import sys
from tqdm import tqdm
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from src.cmesrc.utils import filepaths_updated_swan_data

import matplotlib.pyplot as plt
import numpy as np

In [2]:
SWAN = filepaths_updated_swan_data()


==CACHING SWAN DATA.==



100%|██████████| 4098/4098 [00:00<00:00, 619352.04it/s]

In [3]:
import pandas as pd 

for harpnum, filepath in tqdm(SWAN.items()):

    data = pd.read_csv(filepath, sep="\t")

    plt.close()
    plt.clf()

    fig, ax = plt.subplots(2, figsize=(10,30), num=1, clear=True, sharex=True)

    X = np.arange(len(data), dtype=np.float64)
    lon_min = data["LONDTMIN"].values
    lon_max = data["LONDTMAX"].values
    lat_min = data["LATDTMIN"].values
    lat_max = data["LATDTMAX"].values

    rotated_mask = data["IRBB"].values

    original_x = np.copy(X)
    original_lon_min = np.copy(lon_min)
    original_lon_max = np.copy(lon_max)
    original_lat_min = np.copy(lat_min)
    original_lat_max = np.copy(lat_max)

    original_x[rotated_mask] = np.nan
    original_lon_min[rotated_mask] = np.nan
    original_lon_max[rotated_mask] = np.nan
    original_lat_min[rotated_mask] = np.nan
    original_lat_max[rotated_mask] = np.nan

    rotated_x = np.copy(X)
    rotated_lon_min = np.copy(lon_min)
    rotated_lon_max = np.copy(lon_max)
    rotated_lat_min = np.copy(lat_min)
    rotated_lat_max = np.copy(lat_max)

    rotated_x[~rotated_mask] = np.nan
    rotated_lon_min[~rotated_mask] = np.nan
    rotated_lon_max[~rotated_mask] = np.nan
    rotated_lat_min[~rotated_mask] = np.nan
    rotated_lat_max[~rotated_mask] = np.nan

    ax[0].plot(original_x, original_lon_min, color="black", linestyle="solid")
    ax[0].plot(original_x, original_lon_max, color="orange", linestyle="solid")

    ax[0].plot(rotated_x, rotated_lon_min, color="black", linestyle="dotted")
    ax[0].plot(rotated_x, rotated_lon_max, color="orange", linestyle="dotted")

    ax[0].set_title(f"HARP {harpnum} LON")

    ax[1].plot(original_x, original_lat_min, color="black", linestyle="solid")
    ax[1].plot(original_x, original_lat_max, color="orange", linestyle="solid")

    ax[1].plot(rotated_x, rotated_lat_min, color="black", linestyle="dotted")
    ax[1].plot(rotated_x, rotated_lat_max, color="orange", linestyle="dotted")

    ax[1].set_title(f"HARP {harpnum} LAT")

    plt.savefig(f"figures/bboxes/{harpnum}.png", dpi=150)

    plt.clf()
    plt.cla()
    plt.close()

    fig.clear()
    plt.close(fig)


100%|██████████| 4098/4098 [11:23<00:00,  6.00it/s]


: 

In [11]:
import drms

client = drms.Client(email="ucapern@ucl.ac.uk", verbose=True)
harpnum = 12
hmi_request = f"hmi.sharp_720s[{harpnum}]"

keywords = client.query(hmi_request, key="LAT_MIN, LON_MIN, LAT_MAX, LON_MAX, LONDTMIN, LONDTMAX, LATDTMIN, LATDTMAX, T_REC")

In [23]:
from astropy.time import Time

Time(keywords["T_REC"].apply(lambda x: x[:-4].replace(".","-").replace("_"," ")).to_list())[0]

<Time object: scale='utc' format='iso' value=2010-05-03 05:12:00.000>

: 

In [14]:
ORIGINAL_SWAN[2].iloc[0].name

<Time object: scale='utc' format='iso' value=2010-05-01 00:00:00.000>

In [24]:
keywords

LON_MIN = keywords["LON_MIN"].values
LON_MAX = keywords["LON_MAX"].values
LAT_MIN = keywords["LAT_MIN"].values
LAT_MAX = keywords["LAT_MAX"].values

LONDTMIN = keywords["LONDTMIN"].values
LONDTMAX = keywords["LONDTMAX"].values
LATDTMIN = keywords["LATDTMIN"].values
LATDTMAX = keywords["LATDTMAX"].values


In [25]:
from matplotlib import animation
import matplotlib.patches as patches

fig, ax = plt.subplots()

dtbox = patches.Rectangle(
    (LONDTMIN[0], LATDTMIN[0]),
    LONDTMAX[0] - LONDTMIN[0],
    LATDTMAX[0] - LATDTMIN[0],
    fc="forestgreen",
    zorder=10
)

box = patches.Rectangle(
    (LON_MIN[0], LAT_MIN[0]),
    LON_MAX[0] - LON_MIN[0],
    LAT_MAX[0] - LAT_MIN[0],
    fc="red",
    zorder=20
)

def init():
    ax.add_patch(dtbox)
    ax.add_patch(box)
    return dtbox, box

def animate(i):
    print([LONDTMIN[i], LATDTMIN[i]])
    dtbox.set_xy([LONDTMIN[i], LATDTMIN[i]])
    dtbox.set_width(LONDTMAX[i] - LONDTMIN[i])
    dtbox.set_height(LATDTMAX[i] - LATDTMIN[i])

    box.set_xy([LON_MIN[i], LAT_MIN[i]])
    box.set_width(LON_MAX[i] - LON_MIN[i])
    box.set_height(LAT_MAX[i] - LAT_MIN[i])
    return dtbox, box

ax.set_xlim(-90,90)
ax.set_ylim(-90,90)

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(LON_MIN), interval=100, blit=True)

writer = animation.FFMpegWriter(fps=60)

anim.save("figures/bboxes/animation.mp4", writer=writer, dpi=200)

<IPython.core.display.Javascript object>

[-96.611603, -23.5504]
[-96.500298, -23.5504]
[-96.389, -23.5504]
[-96.277702, -23.5504]
[-96.166397, -23.5504]
[-96.055099, -23.5504]
[-95.943802, -23.5504]
[-95.832497, -23.5504]
[-95.721199, -23.5504]
[-95.609901, -23.5504]
[-95.498596, -23.5504]
[-95.387299, -23.5504]
[-95.276001, -23.5504]
[-95.164703, -23.5504]
[-95.053398, -23.5504]
[-94.942101, -23.5504]
[-94.830803, -23.5504]
[-94.719498, -23.5504]
[-94.6082, -23.5504]
[-94.497002, -23.5504]
[-94.385696, -23.5504]
[-94.274399, -23.5504]
[-94.163101, -23.5504]
[-94.051804, -23.5504]
[-93.940498, -23.5504]
[-93.829201, -23.5504]
[-93.717903, -23.5504]
[-93.606598, -23.5504]
[-93.4953, -23.5504]
[-93.384003, -23.5504]
[-93.272697, -23.5504]
[-93.1614, -23.5504]
[-93.050102, -23.5504]
[-92.938797, -23.5504]
[-92.827499, -23.5504]
[-92.716202, -23.5504]
[-92.604897, -23.5504]
[-92.493599, -23.5504]
[-92.382301, -23.5504]
[-92.271004, -23.5504]
[-92.159698, -23.5504]
[-92.048401, -23.5504]
[-91.937103, -23.5504]
[-91.825798, -23.550

In [34]:
from src.cmesrc.utils import cache_swan_data
from src.cmesrc.config import UPDATED_SWAN
import pandas as pd
from astropy.time import Time
from os.path import join
from tqdm import tqdm
import drms

client = drms.Client(email="ucapern@ucl.ac.uk", verbose=True)

SWAN = cache_swan_data()


==CACHING SWAN DATA.==



0it [00:00, ?it/s]
100%|██████████| 1045/1045 [00:14<00:00, 71.69it/s]

In [61]:
harpnum = 7302

data = SWAN[7302].copy()

df = pd.read_csv(f"/home/julio/cmesrc/data/raw/mvts/SWAN/partition5/{harpnum}.csv", sep="\t")

hmi_request = f"hmi.sharp_720s[{harpnum}]"

keywords = client.query(hmi_request, key="LONDTMIN, LONDTMAX, LATDTMIN, LATDTMAX, T_REC")

In [62]:
data

,Timestamp,TOTUSJH,TOTBSQ,TOTPOT,TOTUSJZ,ABSNJZH,SAVNCPP,USFLUX,TOTFZ,MEANPOT,...,BFLARE_LABEL_LOC,CFLARE_LOC,CFLARE_LABEL_LOC,MFLARE_LOC,MFLARE_LABEL_LOC,XFLARE_LOC,XFLARE_LABEL_LOC,XR_MAX,XR_QUAL,IS_TMFI
Timestamp,,,,,,,,,,,,,,,,,,,,,
2018-09-02 14:24:00.000,2018-09-02 14:24:00.000,9.904866,1.104508e+08,5.222385e+20,2.261780e+11,0.542028,4.049080e+10,2.560856e+20,-1.181900e+23,956.785171,...,None,0.0,None,0.0,None,0.0,None,7.796200e-09,12,False
2018-09-02 14:36:00.000,2018-09-02 14:36:00.000,11.066074,1.314481e+08,6.209726e+20,2.779712e+11,0.601328,4.065440e+10,3.203495e+20,-1.384647e+23,845.541105,...,None,0.0,None,0.0,None,0.0,None,1.102900e-08,12,False
2018-09-02 14:48:00.000,2018-09-02 14:48:00.000,12.221666,1.397565e+08,6.510724e+20,2.979577e+11,0.525728,1.019194e+10,3.369294e+20,-1.458221e+23,840.907070,...,None,0.0,None,0.0,None,0.0,None,1.129100e-08,12,False
2018-09-02 15:00:00.000,2018-09-02 15:00:00.000,13.585913,1.551827e+08,7.244791e+20,3.372195e+11,0.211692,9.888083e+10,3.731820e+20,-1.612290e+23,837.977203,...,None,0.0,None,0.0,None,0.0,None,7.022300e-09,12,False
2018-09-02 15:12:00.000,2018-09-02 15:12:00.000,15.677431,1.654935e+08,7.916320e+20,3.723830e+11,0.321354,1.300179e+11,4.004776e+20,-1.729090e+23,849.128896,...,None,0.0,None,0.0,None,0.0,None,1.015400e-08,12,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-09-04 23:12:00.000,2018-09-04 23:12:00.000,23.924030,1.703402e+08,1.291694e+21,5.878995e+11,4.166707,3.119805e+11,3.818934e+20,-1.334652e+23,1407.566950,...,None,0.0,None,0.0,None,0.0,None,6.870200e-09,12,False
2018-09-04 23:24:00.000,2018-09-04 23:24:00.000,27.011481,1.768467e+08,1.379798e+21,6.742953e+11,3.821889,2.861183e+11,3.919202e+20,-1.363893e+23,1453.103983,...,None,0.0,None,0.0,None,0.0,None,7.344000e-09,12,False
2018-09-04 23:36:00.000,2018-09-04 23:36:00.000,27.906310,1.731557e+08,1.421519e+21,6.897699e+11,3.577958,2.867322e+11,3.866092e+20,-1.338836e+23,1522.596574,...,None,0.0,None,0.0,None,0.0,None,4.548000e-09,12,False


In [63]:
keywords["Timestamp"] = Time(keywords["T_REC"].apply(lambda x: x[:-4].replace(".","-").replace("_"," ")).to_list())
keywords.drop(columns=["T_REC"], inplace=True)

In [64]:
# data.drop(columns=["Timestamp"], inplace=True)

In [65]:
data = pd.merge(data, keywords, on="Timestamp", how="outer")

ValueError: 'Timestamp' is both an index level and a column label, which is ambiguous.

: 

In [58]:
data.to_csv("test.csv", index=False)

In [59]:
df = pd.read_csv("test.csv")

In [60]:
df

/home/julio/cmesrc/env/lib/python3.9/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/home/julio/cmesrc/env/lib/python3.9/site-packages/IPython/core/formatters.py:344: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,Timestamp,TOTUSJH,TOTBSQ,TOTPOT,TOTUSJZ,ABSNJZH,SAVNCPP,USFLUX,TOTFZ,MEANPOT,...,MFLARE_LABEL_LOC,XFLARE_LOC,XFLARE_LABEL_LOC,XR_MAX,XR_QUAL,IS_TMFI,LONDTMIN,LONDTMAX,LATDTMIN,LATDTMAX
0,2018-09-02 14:24:00.000,9.904866,1.104508e+08,5.222385e+20,2.261780e+11,0.542028,4.049080e+10,2.560856e+20,-1.181900e+23,956.785171,...,None,0.0,None,7.796200e-09,12.0,False,-60.531200,-53.358200,0.9829,4.821
1,2018-09-02 14:36:00.000,11.066074,1.314481e+08,6.209726e+20,2.779712e+11,0.601328,4.065440e+10,3.203495e+20,-1.384647e+23,845.541105,...,None,0.0,None,1.102900e-08,12.0,False,-60.417801,-53.244801,0.9829,4.821
2,2018-09-02 14:48:00.000,12.221666,1.397565e+08,6.510724e+20,2.979577e+11,0.525728,1.019194e+10,3.369294e+20,-1.458221e+23,840.907070,...,None,0.0,None,1.129100e-08,12.0,False,-60.304401,-53.131401,0.9829,4.821
3,2018-09-02 15:00:00.000,13.585913,1.551827e+08,7.244791e+20,3.372195e+11,0.211692,9.888083e+10,3.731820e+20,-1.612290e+23,837.977203,...,None,0.0,None,7.022300e-09,12.0,False,-60.191002,-53.018002,0.9829,4.821
4,2018-09-02 15:12:00.000,15.677431,1.654935e+08,7.916320e+20,3.723830e+11,0.321354,1.300179e+11,4.004776e+20,-1.729090e+23,849.128896,...,None,0.0,None,1.015400e-08,12.0,False,-60.077702,-52.904598,0.9829,4.821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,2018-09-10 00:36:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,40.507301,47.680401,0.9829,4.821
847,2018-09-10 00:48:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,40.620701,47.793800,0.9829,4.821
848,2018-09-10 01:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,40.734100,47.907200,0.9829,4.821
849,2018-09-10 01:12:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,40.847500,48.020599,0.9829,4.821
